In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import json
from time import time
import pickle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers.merge import add

Using TensorFlow backend.


In [4]:
model = load_model("D:/Machine Learning/WMC project/Image Captioning/model_weights/model_9.h5")





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where












In [5]:
model_temp = ResNet50(weights="imagenet",input_shape=(224,224,3))

In [6]:
model_resnet = Model(model_temp.input,model_temp.layers[-2].output)

In [7]:
def preprocess_img(img):
    img = image.load_img(img,target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    # Normalisation
    img = preprocess_input(img)
    return img

In [8]:
def encode_image(img):
    img = preprocess_img(img)
    feature_vector = model_resnet.predict(img)
    
    feature_vector = feature_vector.reshape(1,feature_vector.shape[1])
    #print(feature_vector.shape)
    return feature_vector

In [10]:
with open("D:/Machine Learning/WMC project/Image Captioning/storage/word_to_idx.pkl", "rb") as w2i:
    word_to_idx=pickle.load(w2i);
with open("D:/Machine Learning/WMC project/Image Captioning/storage/idx_to_word.pkl", "rb") as i2w:
    idx_to_word=pickle.load(i2w);   

In [15]:
def predict_caption(photo):
    
    in_text = "startseq"
    max_len=35
    for i in range(max_len):
        sequence = [word_to_idx[w] for w in in_text.split() if w in word_to_idx]
        sequence = pad_sequences([sequence],maxlen=max_len,padding='post')
        
        ypred = model.predict([photo,sequence])
        ypred = ypred.argmax() #WOrd with max prob always - Greedy Sampling
        word = idx_to_word[ypred]
        in_text += (' ' + word)
        
        if word == "endseq":
            break
    
    final_caption = in_text.split()[1:-1]
    final_caption = ' '.join(final_caption)
    return final_caption

In [13]:
enc = encode_image("pic1.jpg")

In [16]:
predict_caption(enc)

'man in red coat is standing in the snow'

In [3]:
dic1={}
dic1['image']="imagepath"
dic1['caption']="captionpath"

In [4]:
dic2=dic1

In [5]:
dic2['image']

'imagepath'